**Background**

Brain recordings, such as EEG and fMRI seek to characterize brain activity and connectivity. A complete understanding of the braib dynamics can lead to a much better understanding of brain pathologies and can help direct new paths into AI. Recordings seek to determine functional connectivity between the different hierarchial organizations of the brain, and this ideal representation of the brain is very important, because it can be directly mapped to brain physiology and it can directly inform us of the dynamics residing in the brain structure.

Specifically, $ \begin{equation} S = M(E(B(N,I,t))) \end{equation}$ , where S is our recorded signal, B is the functional representation of the brain, dependent on N, the brain network, I, the input to the brain, and t, time. E is representing the mapping from the functional representation to the actual electrophysiology that encodes that information. M is representing the mapping from brain electrophysiology to the measurements achieved by recording devices.

Directed Brain Network $ \begin{equation} N = (\vec{v},\vec{e}) \end{equation}$, where v correspond to vertice states, and e correspond to edge functions between vertices. v is multidimensional, can represent many hierarchial levels in the brain, such as neurons, nerve bundles, and whole regions. For neurons, the state could take into many factors, for example, location, topology of the neuron, phase/frequency of firing, number of synaptic vesicles etc.
Brain Dynamics B: $ \begin{equation} \dot{v_{i}} = g(I) + \sum_{e \in N}{e(v_{j},v_{i},t)} \end{equation}$

*Note that there can be self directed loops where edge function* = $ \begin{equation} e(v_{i},t) \end{equation}$

The Input signal can encapsulate input signal from the environment and internal signaling. Signals from the environement can be more directly measured (visual cues, sensory features etc.) and internal signaling is less direct, but both contain some noise and lost information.

Electrophysiology E(B) maps the network state into a corresponding voltage density topology. Several electrophysiology models already exist to predict mapping of brain state to electrophysiology.

M(E) is takes the measurements from the eletrophysiology. Almost total information is known about M, since it's fairly easy to measure quantities about M (location of electrodes, orientation etc.) and there is much prior information on M (hardware features, effects of filtering, postprocessing signal etc.). However this is where noise gets introduced, as devices can pick up un extraneuous or faulty information sources.

Since the mapping from the functional brain to electrophysiology is fairly direct, and the functional network intrinsically contains much of the information of the brain system, the general Bayesian problem is as follows:
   
   *Given S,I, find N.*

   However, this is a very hard problem to solve, as S and I are relatively low dimensional signals compared to N. This problem can be reworded into:
    
   *Given S,I, find N' where arg min C(N,N')*

C is our cost function associated with our predicted and actual N , which can encapsulate closeness of behavior of the two systems. Since biological mechanisms are usually repetitive and fractal-like, giving rise to complex structures with relatively simple rulesets and the brain can be globally modulated by input signals such as drugs, DBS, and rTMS, there may be some low dimensional features that can be extracted from S that can be used to predict the behavior of N to relatively high degree.

There already exist multitudes of signal features that the current literature utilizes to ascertain certain features of N, but efficacy of these features to predict functional behavior is quantitaively uncertain, and should be studies further to determine relations between the detected features and underlying brain network behavior. Underlied here is a path to quantitavely determine upper bounds on these features in describing functional network behavior.
    
A simple, ideal brain network B(N) with I = 0, and the network being time-invariant, was contructed. Then E was assumed to be identity functions, M was assumed to be a linear function and S was the actual signal from the time series of $ \begin{equation} \vec{v} \end{equation}$. Several measures curently used in the literatrure were then tested in a Monte Carlo simulation to get upper bounds of the efficacy. The constructed model is as follows:

$\begin{equation}
\dot{v_{i}} = g(v_{i}) + \sum_{e \in N}{f(v_{j},v_{i})}
\end{equation}$

$ \begin{equation}
S = M \vec{v} + N
\end{equation}$
,where N is noise
    
   Network: All-to-All (K), Small-World (SW), Preferential Attachment (BA)

**Correlation Measures**
    
CFC: PPC, AAC, PAC

Coherence: CTC, PLV

Spectral: RDP, Edge, Entropy, Moment

Nonlinear: Correlation Dimension/Saturation, Largest Lyapunov exponent, Kolmogorof entropy

Mutual Entropy: CMI, AMI

Connectivity Measures: TE, GCI, PLI, Pairwise Phase Consistency, GC, DT, PDC

**Documentation**

Classes:
- nmodel:
    - VARIABLES:
        - G: graph network
        - x: state vector (array) [node,vector_state,time_point]
        - h: node function R^n -> R^n
        - f: coupling function R^n -> R^n
        - M: measurement matrix (matrix)
        - N: variance for Gaussian noise
        - y: measurement vector (array)
        - t: time
        - dt: time step
    - FUNCTIONS:
        - dev: IN: state, OUT: returns derivative values (array)
        - linear_measure: IN: none, OUT: returns measurement values (array)
        - euler_step: IN: none, OUT: none, euler approximation step
        - runge_kutta_step: IN: none, OUT: none, 4th order runge-kutta approximation step
        - step: IN: none, OUT: none, step call
        - run: IN: T, OUT: none, runs model for time T
        - clear_run: IN: none, OUT: none, clears all states exept initial

Functions:
- create_states: IN: a, b, c(if complex) = (a',b'), distribution('normal','logistic','uniform')default is point, OUT: states
- create_vec_states: IN: array of tuples for parameters for create_states, OUT: states
- create_network: IN: n number of nodes, network type('SW','BA','SCC','K'), parameters (k,p,m), OUT: graph G
- multiply_graph: IN: graph G , side, nodal, edge, other, OUT: graph G' multiplied by side
- plt_graph: IN: graph G, OUT: plot of adjacency matrix
- state_course: IN: states (matrix), OUT: plot of states
- spectrogram: IN: signal, fs, window, nperseg, noverlap, nfft, OUT: plot of spectrogram
- PSD: IN: singal, fs, window, nperseg, noverlap, nfft, OUT: plot of PSD
- cross_func: IN: states (matrix), func, OUT: matrix of coefficients
- reduce_state: IN: states (array), OUT: slice of states with only one vector part (matrix)
- mutual_info_score: IN:  OUT:
- phase_locking_value: IN: x,y OUT: phase locking value
- coh: IN: x,y OUT: coherence value
- cor: IN: x,y OUT: correlation value

In [2]:
#RUN THIS BEFORE USING

%reset
%matplotlib inline
import network_modules as nm
import networkx as nx
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import cmath as c

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [3]:
#Initialization

distributions = [(6,1,1,'normal'),(6,2),(6,1,1,'logistic'),(6,0,6,'uniform')]
x = nm.create_vec_states(distributions)          

G = nm.create_network('SCC') #Basic Graph

def h(x):   #node function
    return 0

def f(x,y): #coupling function
    return y-x

M = np.identity(6) #Measurement Matrix

N = 1 #Noise Variance

#Initialize & Run model for t = 10 sec
nmod = nm.nmodel(G,x,h,f,M,N)
nmod.run(10)

[[-0.46914659  2.          5.33839038  0.19135434]
 [ 1.7284028   2.          0.96131355  5.09559561]
 [ 2.57399994  2.          2.55153026  2.23514634]
 [ 1.41404189  2.          2.22960408  0.54709856]
 [ 1.6548625   2.          3.63221585  1.97262859]
 [ 1.42114664  2.          0.85880901  4.28408697]]


In [3]:
y = nm.reduce_state(0, nmod.x)
nm.cross_func(y,nm.phase_locking_value)

matrix([[ 1.        ,  0.99179288,  0.97552939,  0.95667491,  0.86287535,
          0.96550983],
        [ 0.99179288,  1.        ,  0.99000881,  0.95909508,  0.83580317,
          0.96669234],
        [ 0.97552939,  0.99000881,  1.        ,  0.9775481 ,  0.85582013,
          0.98142522],
        [ 0.95667491,  0.95909508,  0.9775481 ,  1.        ,  0.91456535,
          0.99925961],
        [ 0.86287535,  0.83580317,  0.85582013,  0.91456535,  1.        ,
          0.90743555],
        [ 0.96550983,  0.96669234,  0.98142522,  0.99925961,  0.90743555,
          1.        ]])